In [2]:
import numpy as np
import pandas as pd

In [4]:
df=pd.read_excel("/content/Solar Dataset_Solar from MEDA-1.XLSX")

In [5]:
df.dropna(inplace=True)

In [6]:
df=df[[ '#date time', 'el','az','ghi','wb','precip','press','dp','rh','dni','ws','maxws','airtemp']]

In [7]:
df.replace(' NaN',0,inplace=True)

In [10]:
#writing a facebook prophet model to predict dni using all features #date time', 'el','az','ghi','wb','precip','press','dp','rh','dni','ws','maxws','airtemp

from prophet import Prophet
df_prophet = df.rename(columns={'#date time': 'ds', 'dni': 'y'})
model = Prophet(
    growth='linear',
    changepoints=['2020-01-01'],
    n_changepoints=1,
    yearly_seasonality=True,
    weekly_seasonality=True,
    daily_seasonality=True,
    seasonality_mode='additive',
    holidays=None,
    seasonality_prior_scale=10.0,
    changepoint_prior_scale=0.5,
    holidays_prior_scale=10.0,
    mcmc_samples=0,
    interval_width=0.80,
    uncertainty_samples=1000,
)
model.add_regressor('el')
model.add_regressor('az')
model.add_regressor('ghi')
model.add_regressor('wb')
model.add_regressor('precip')
model.add_regressor('press')
model.add_regressor('dp')
model.add_regressor('rh')
model.add_regressor('ws')
model.add_regressor('maxws')
model.add_regressor('airtemp')
model.fit(df_prophet)


DEBUG:cmdstanpy:input tempfile: /tmp/tmpw6jf9vg3/xbj5rbwu.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpw6jf9vg3/uz52mj1p.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=20778', 'data', 'file=/tmp/tmpw6jf9vg3/xbj5rbwu.json', 'init=/tmp/tmpw6jf9vg3/uz52mj1p.json', 'output', 'file=/tmp/tmpw6jf9vg3/prophet_model45h9z1si/prophet_model-20240317140701.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
14:07:01 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
14:07:05 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


In [11]:
#evaluating above model

import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Make predictions
y_pred = model.predict(df_prophet)

# Calculate and print metrics
print('Mean Squared Error (MSE):', mean_squared_error(df_prophet['y'], y_pred['yhat']))
print('Mean Absolute Error (MAE):', mean_absolute_error(df_prophet['y'], y_pred['yhat']))
print('Root Mean Squared Error (RMSE):', np.sqrt(mean_squared_error(df_prophet['y'], y_pred['yhat'])))
print('R-squared:', r2_score(df_prophet['y'], y_pred['yhat']))


Mean Squared Error (MSE): 13601.965896668748
Mean Absolute Error (MAE): 85.97317200399613
Root Mean Squared Error (RMSE): 116.6274663047635
R-squared: 0.6890022480686021


In [15]:

import pandas as pd
# Predict the next 5 values of DNI
future_dates = pd.date_range(start='2023-01-01', periods=5, freq='D')
future_df = pd.DataFrame({'ds': future_dates})
future_df['el'] = df['el'].iloc[-1]
future_df['az'] = df['az'].iloc[-1]
future_df['ghi'] = df['ghi'].iloc[-1]
future_df['wb'] = df['wb'].iloc[-1]
future_df['precip'] = df['precip'].iloc[-1]
future_df['press'] = df['press'].iloc[-1]
future_df['dp'] = df['dp'].iloc[-1]
future_df['rh'] = df['rh'].iloc[-1]
future_df['ws'] = df['ws'].iloc[-1]
future_df['maxws'] = df['maxws'].iloc[-1]
future_df['airtemp'] = df['airtemp'].iloc[-1]
future_pred = model.predict(future_df)

# Print the actual and predicted DNI values
print('Actual DNI values:')
print(df['dni'].tail(5))
print('\nPredicted DNI values:')
print(future_pred['yhat'].head(5))


Actual DNI values:
52767    224
52768    100
52769      9
52770    237
52771    314
Name: dni, dtype: int64

Predicted DNI values:
0   -190.310684
1   -187.311292
2   -180.550328
3   -178.657860
4   -180.694449
Name: yhat, dtype: float64
